O intuito desse app é automatizar as informações para o processo de criação de notas de débito do time de operações.

In [2]:
from cdv_tools.gcp_clients.gsheet import GSheet_Client
from cdv_tools.portfolio_cdvcom.data_accessor import DataAccessor
from google.cloud import storage
import openpyxl as pyxl
import os
import pandas as pd
import datetime as dt
from datetime import date, timedelta
import numpy as np
import requests
import json
from nd import gerar_nota

In [3]:
def get_records_from_sheetFiltro():
    sheet_id = '1WoCbw1tCk3vQ8Z8Vfbld4oukQoPmaa13GEGNA55GVNE'
    #saccount_email = 'get-var-info@portfolio-comercializadora.iam.gserviceaccount.com'
    saccount_email = 'default-cf@portfolio-comercializadora.iam.gserviceaccount.com'

    gsheet_client = GSheet_Client(saccount_email=saccount_email, default_spreadsheet_id=sheet_id)

    data = gsheet_client.sheets.get_by_sheet_name(sheet_name='Filtro_Email_A RECEBER')
    
    # for column in data.columns:
    #     if column  in ['Parte - Código do Perfil', 'ContraParte - Código do Perfil', 'RETUSD', 'TUSD CONTRATO', 'TUSD APURADA CCEE', 'COBRAR']:
    #         data[column] = pd.to_numeric(data[column]).isnull()


    df = pd.DataFrame(data)
    
    return df

heets = get_records_from_sheetFiltro()
heets = heets[heets["COBRAR\nR$"].astype(str) != 'None']

print(heets)

          Operação Entrega Compra\n/Venda           Parte - Perfil  \
1      CINE5001-23       1         Compra  CASA DOS VENTOS COM INE   
2      CINE5003-24       1         Compra  CASA DOS VENTOS COM INE   
3        CI5039-24       1         Compra   CASA DOS VENTOS COM I5   
4        CI5042-24       1         Compra   CASA DOS VENTOS COM I5   
5   IC-CINE5043-23       1         Compra                 CVER INE   
6        CI5006-24       1         Compra   CASA DOS VENTOS COM I5   
7        CI5008-24       1         Compra   CASA DOS VENTOS COM I5   
8        CI5010-24       1         Compra   CASA DOS VENTOS COM I5   
9        CI5009-24       1         Compra   CASA DOS VENTOS COM I5   
10     CINE5001-24       1         Compra  CASA DOS VENTOS COM INE   
11        SX001-23       1         Compra  CASA DOS VENTOS COM INE   
12       CI5012-24       1         Compra   CASA DOS VENTOS COM I5   
13       CI5011-24       1         Compra   CASA DOS VENTOS COM I5   
14       CI5034-24  

Ler bases do thunders, da ccee e de pagamentos

In [4]:
def get_records_from_sheet():
    sheet_id = '1WoCbw1tCk3vQ8Z8Vfbld4oukQoPmaa13GEGNA55GVNE'
    #saccount_email = 'get-var-info@portfolio-comercializadora.iam.gserviceaccount.com'
    saccount_email = 'default-cf@portfolio-comercializadora.iam.gserviceaccount.com'

    gsheet_client = GSheet_Client(saccount_email=saccount_email, default_spreadsheet_id=sheet_id)

    data = gsheet_client.sheets.get_by_sheet_name(sheet_name='A RECEBER PY')
    
    # for column in data.columns:
    #     if column  in ['Parte - Código do Perfil', 'ContraParte - Código do Perfil', 'RETUSD', 'TUSD CONTRATO', 'TUSD APURADA CCEE', 'COBRAR']:
    #         data[column] = pd.to_numeric(data[column]).isnull()


    df = pd.DataFrame(data)
    
    return df

def get_records_from_sheet_SAP():
    sheet_id = '1lp3s2-hIvEh_rU1fJUrdge_HNTYaPYUeyJdRRo43SWU'
    #saccount_email = 'get-var-info@portfolio-comercializadora.iam.gserviceaccount.com'
    saccount_email = 'default-cf@portfolio-comercializadora.iam.gserviceaccount.com'

    gsheet_client = GSheet_Client(saccount_email=saccount_email, default_spreadsheet_id=sheet_id)

    data = gsheet_client.sheets.get_by_sheet_name(sheet_name='SAP')
    
    # for column in data.columns:
    #     if column  in ['Parte - Código do Perfil', 'ContraParte - Código do Perfil', 'RETUSD', 'TUSD CONTRATO', 'TUSD APURADA CCEE', 'COBRAR']:
    #         data[column] = pd.to_numeric(data[column]).isnull()


    df = pd.DataFrame(data)
    
    return df

def get_thunders_data(gcs_client,file_prefix):
    

    price_bucket = gcs_client.bucket('thunders-operations')

    blob_name = f"current.csv"
    blob = price_bucket.blob(blob_name)
    blob.reload()

    filename = file_prefix+"thuders_data.csv"
    blob.download_to_filename(filename)
    col_read = ['partyProfileCode', 'counterpartyProfileCode','partyCNPJ','counterpartyCNPJ']
    df = pd.read_csv(filename, sep=";",usecols=col_read)
   
    return df

def get_and_modify_xl(base):

    # load excel file
    for j in range(0,base.shape[0]):
        workbook = pyxl.load_workbook(filename="solicitacao.xlsx")

        sheet = workbook['ND001']

        for i in range(13,20):
            sheet[f"C{i}"] = base.iat[j,(i-13)]
        
        for i in range (24,27):
            sheet[f"C{i}"] = base.iat[j,(i-16)]

        workbook.save(filename=f"ND {j} - {base['file_name'].iloc[j]}.xlsx")
        
    return 0
   

In [54]:
# for _,row in base.iterrows():
#    select = row

#    get_and_modify_xl(row,filename) 

In [55]:
# for _, row in df.iterrows():
#         row["a"] = 

In [7]:
# Gerando as bases que precisamos pra gerar a nota

    # 1) Base do SAP

sap_ext = get_records_from_sheet_SAP()

sap_ext['Nome 2'] = sap_ext['Nome 2'].fillna("")

sap_ext['Nome'] = sap_ext['Nome 1'].apply(str).str.cat(sap_ext['Nome 2'],sep = " ")


sap = sap_ext[['Nome','Local', 'Bairro', 'Rua', 'Código postal', 'Região', 'Telefone 1', 'Nº ID fiscal 1']]

rename_col = {'Nome': 'Nome',
              'Local': 'Cidade',
              'Bairro': 'Bairro',
              'Rua': 'Endereço',
              'Código postal':'CEP',
              'Região': 'UF',
              'Telefone 1': 'Telefone',
              'Nº ID fiscal 1':'CNPJ'}

sap = sap.rename(columns=rename_col)

sap['CNPJ'] = sap['CNPJ'].astype("str")

CNPJ = pd.DataFrame()

CNPJ['pt1'] = sap['CNPJ'].str.slice(0,2)
CNPJ['pt2'] = sap['CNPJ'].str.slice(2,5)
CNPJ['pt3'] = sap['CNPJ'].str.slice(5,8)
CNPJ['pt4'] = sap['CNPJ'].str.slice(8,12)
CNPJ['pt5'] = sap['CNPJ'].str.slice(12,14)

sap['CNPJ'] = (CNPJ['pt1'].str.cat([CNPJ['pt2'],CNPJ['pt3']],sep='.')).str.cat(CNPJ['pt4'],sep="/").str.cat(CNPJ['pt5'],sep='-')

sap = sap.rename(columns=rename_col)

sap['Telefone'] = sap['Telefone'].fillna(0)

sap['Telefone'] = sap['Telefone'].round(0)

sap['Telefone'] = sap['Telefone'].astype("str")

sap['Telefone'] = sap['Telefone'].str.slice(0,-2)


sap.to_csv('ble.csv',index = False)

In [8]:
# Pegando a base do Thunders para conseguirmos os cnpjs das empresas 

gcs_client = storage.Client(project='portfolio-comercializadora')


In [9]:
# Base de dados de sheets

sheets = get_records_from_sheetFiltro()

#print(sheets["COBRAR\nR$"])
sheets = sheets[sheets["COBRAR\nR$"].astype(str) != 'None']
#print(sheets["COBRAR\nR$"])

sheets["COBRAR\nR$"] = sheets["COBRAR\nR$"].apply(str).str.replace(",",".")

#print(sheets["COBRAR\nR$"])

sheets["COBRAR\nR$"] = pd.to_numeric(sheets["COBRAR\nR$"])

sheets["Parte - Código do Perfil"] = pd.to_numeric(sheets["Parte - Código do Perfil"])

sheets["ContraParte - Código do Perfil"] = pd.to_numeric(sheets["ContraParte - Código do Perfil"])

sheets["COBRAR\nR$"]= sheets["COBRAR\nR$"].round(2)

sheets = sheets[sheets["RECEBIDO"].isnull()]

sheets = sheets[sheets["COBRADO"].isnull()]

#sheets["COBRADO"] = pd.to_datetime(sheets["COBRADO"], errors='coerce', format='%d/%m/%Y')

#data_comparacao = datetime(date.today().day,date.today().month,date.today().year) - timedelta(days=30)

#sheets = sheets[sheets["COBRADO"] < data_comparacao]

base = sheets.copy().reset_index(drop= True)

base = base[base["COBRAR\nR$"]>0]

bf = base.copy().reset_index(drop = True)

bf = bf[['Parte - Perfil','Parte - Código do Perfil','Contraparte - Perfil','ContraParte - Código do Perfil','Mês','CNPJ Parte','CNPJ Contraparte','COBRAR\nR$']]
bf = bf.sort_values(by=['Parte - Código do Perfil','ContraParte - Código do Perfil'])
code_list= [27977, 28024, 50609, 51810, 49254]
bf = bf[bf['Parte - Código do Perfil'].isin(code_list)]
bf = bf[-bf['ContraParte - Código do Perfil'].isin(code_list)]

print(bf)



             Parte - Perfil  Parte - Código do Perfil  \
3    CASA DOS VENTOS COM I5                     27977   
5    CASA DOS VENTOS COM I5                     27977   
6    CASA DOS VENTOS COM I5                     27977   
7    CASA DOS VENTOS COM I5                     27977   
8    CASA DOS VENTOS COM I5                     27977   
2    CASA DOS VENTOS COM I5                     27977   
11   CASA DOS VENTOS COM I5                     27977   
12   CASA DOS VENTOS COM I5                     27977   
13   CASA DOS VENTOS COM I5                     27977   
14   CASA DOS VENTOS COM I5                     27977   
15   CASA DOS VENTOS COM I5                     27977   
16   CASA DOS VENTOS COM I5                     27977   
22   CASA DOS VENTOS COM I5                     27977   
21   CASA DOS VENTOS COM I5                     27977   
19   CASA DOS VENTOS COM I5                     27977   
20   CASA DOS VENTOS COM I5                     27977   
18   CASA DOS VENTOS COM I5    

In [10]:
sheets[sheets["COBRAR\nR$"] != 'None']

,Operação,Entrega,Compra\n/Venda,Parte - Perfil,Parte - Código do Perfil,Contraparte - Perfil,ContraParte - Código do Perfil,Fonte de Energia,Início de fornecimento,Fim do fornecimento,...,CONTABILIZAÇÂO,COBRADO,RECEBIDO,key2,found?,volume igual?,retusd igual?,tusd apurada igual?,cobrar igual?,Cobrado
1,CINE5001-23,1,Compra,CASA DOS VENTOS COM INE,28024,ENERGISA CQ I5,70102,Incentivada Não Especial 50%,01/08/2023,31/12/2038,...,2024_05 - APURAÇÃO DO DESCONTO NA TUSD/TUST,None,None,None,None,None,None,None,None,None
2,CINE5003-24,1,Compra,CASA DOS VENTOS COM INE,28024,ENERGISA CQ I5,70102,Incentivada Não Especial 50%,01/01/2024,31/12/2039,...,2024_05 - APURAÇÃO DO DESCONTO NA TUSD/TUST,None,None,None,None,None,None,None,None,None
3,CI5039-24,1,Compra,CASA DOS VENTOS COM I5,27977,STATKRAFT I5,11894,Incentivada 50%,01/05/2024,31/05/2024,...,2024_05 - APURAÇÃO DO DESCONTO NA TUSD/TUST,None,None,None,None,None,None,None,None,None
4,CI5042-24,1,Compra,CASA DOS VENTOS COM I5,27977,DEAL COMERCIALIZADORA I5,3726,Incentivada 50%,01/05/2024,31/05/2024,...,2024_05 - APURAÇÃO DO DESCONTO NA TUSD/TUST,None,None,None,None,None,None,None,None,None
5,IC-CINE5043-23,1,Compra,CVER INE,98040,CVER INE,98040,Incentivada Não Especial 50%,01/08/2023,31/12/2031,...,2024_05 - APURAÇÃO DO DESCONTO NA TUSD/TUST,None,None,None,None,None,None,None,None,None
6,CI5006-24,1,Compra,CASA DOS VENTOS COM I5,27977,SOLENERGIAS I5,4571,Incentivada 50%,01/01/2024,31/12/2024,...,2024_05 - APURAÇÃO DO DESCONTO NA TUSD/TUST,None,None,None,None,None,None,None,None,None
7,CI5008-24,1,Compra,CASA DOS VENTOS COM I5,27977,SOLENERGIAS I5,4571,Incentivada 50%,01/01/2024,31/12/2024,...,2024_05 - APURAÇÃO DO DESCONTO NA TUSD/TUST,None,None,None,None,None,None,None,None,None
8,CI5010-24,1,Compra,CASA DOS VENTOS COM I5,27977,SOLENERGIAS I5,4571,Incentivada 50%,01/01/2024,31/12/2024,...,2024_05 - APURAÇÃO DO DESCONTO NA TUSD/TUST,None,None,None,None,None,None,None,None,None
9,CI5009-24,1,Compra,CASA DOS VENTOS COM I5,27977,SOLENERGIAS I5,4571,Incentivada 50%,01/01/2024,31/12/2024,...,2024_05 - APURAÇÃO DO DESCONTO NA TUSD/TUST,None,None,None,None,None,None,None,None,None
10,CINE5001-24,1,Compra,CASA DOS VENTOS COM INE,28024,COPEL COM I5 NESP,28388,Incentivada Não Especial 50%,01/01/2024,31/12/2038,...,2024_05 - APURAÇÃO DO DESCONTO NA TUSD/TUST,None,None,None,None,None,None,None,None,None


In [42]:
sheets

,Operação,Entrega,Compra\n/Venda,Parte - Perfil,Parte - Código do Perfil,Contraparte - Perfil,ContraParte - Código do Perfil,Fonte de Energia,Início de fornecimento,Fim do fornecimento,...,CONTABILIZAÇÂO,COBRADO,RECEBIDO,key2,found?,volume igual?,retusd igual?,tusd apurada igual?,cobrar igual?,Cobrado
893,SX001-23,1,Compra,CASA DOS VENTOS COM INE,28024,COPEL COM I5 NESP,28388,Incentivada Não Especial 50%,01/09/2023,31/12/2035,...,2023_09 - 1ª REAPURAÇÃO DE DESCONTO,2023-11-28,None,None,None,None,None,None,None,None


In [11]:
df = bf[['Parte - Perfil','Parte - Código do Perfil','Contraparte - Perfil','ContraParte - Código do Perfil','Mês','CNPJ Parte','CNPJ Contraparte',]].drop_duplicates().copy().reset_index(drop = True)
code_list= [27977, 28024, 50609, 51810, 49254]
df = df[df['Parte - Código do Perfil'].isin(code_list)]
df = df[-df['ContraParte - Código do Perfil'].isin(code_list)]
print(df)

             Parte - Perfil  Parte - Código do Perfil  \
0    CASA DOS VENTOS COM I5                     27977   
1    CASA DOS VENTOS COM I5                     27977   
2    CASA DOS VENTOS COM I5                     27977   
3    CASA DOS VENTOS COM I5                     27977   
4    CASA DOS VENTOS COM I5                     27977   
5    CASA DOS VENTOS COM I5                     27977   
6    CASA DOS VENTOS COM I5                     27977   
7    CASA DOS VENTOS COM I5                     27977   
8    CASA DOS VENTOS COM I5                     27977   
9    CASA DOS VENTOS COM I5                     27977   
10   CASA DOS VENTOS COM I5                     27977   
11   CASA DOS VENTOS COM I5                     27977   
12   CASA DOS VENTOS COM I5                     27977   
13  CASA DOS VENTOS COM INE                     28024   
14  CASA DOS VENTOS COM INE                     28024   

        Contraparte - Perfil  ContraParte - Código do Perfil      Mês  \
0   DEAL COMER

In [12]:
bf2 = pd.DataFrame(bf.groupby(['Parte - Perfil','Parte - Código do Perfil','Contraparte - Perfil','ContraParte - Código do Perfil'])["COBRAR\nR$"].agg(['sum']))

bf2 = bf2[bf2["sum"]>=10]

merged = pd.merge(bf2,df,how='inner',on=['Parte - Código do Perfil','ContraParte - Código do Perfil'])

merged2 = pd.merge(bf,merged,how = 'inner', on=['Parte - Perfil','Parte - Código do Perfil','Contraparte - Perfil','ContraParte - Código do Perfil','Mês'])

merged2 = merged2.rename(columns={'sum':'Total'})

merged2.head()

,Parte - Perfil,Parte - Código do Perfil,Contraparte - Perfil,ContraParte - Código do Perfil,Mês,CNPJ Parte_x,CNPJ Contraparte_x,COBRAR\nR$,Total,CNPJ Parte_y,CNPJ Contraparte_y
0,CASA DOS VENTOS COM I5,27977,DEAL COMERCIALIZADORA I5,3726,mai.-24,33.933.760/0001-00,10.671.322/0001-16,43.16,43.16,33.933.760/0001-00,10.671.322/0001-16
1,CASA DOS VENTOS COM I5,27977,SOLENERGIAS I5,4571,mai.-24,33.933.760/0001-00,13.459.301/0001-20,469.78,3288.44,33.933.760/0001-00,13.459.301/0001-20
2,CASA DOS VENTOS COM I5,27977,SOLENERGIAS I5,4571,mai.-24,33.933.760/0001-00,13.459.301/0001-20,939.55,3288.44,33.933.760/0001-00,13.459.301/0001-20
3,CASA DOS VENTOS COM I5,27977,SOLENERGIAS I5,4571,mai.-24,33.933.760/0001-00,13.459.301/0001-20,1409.33,3288.44,33.933.760/0001-00,13.459.301/0001-20
4,CASA DOS VENTOS COM I5,27977,SOLENERGIAS I5,4571,mai.-24,33.933.760/0001-00,13.459.301/0001-20,469.78,3288.44,33.933.760/0001-00,13.459.301/0001-20


In [13]:

total_cobranca = merged2[['Contraparte - Perfil','ContraParte - Código do Perfil',"CNPJ Contraparte_x",'Total']]
#total_cobranca= pd.DataFrame(total_cobranca.groupby(['Contraparte - Perfil','ContraParte - Código do Perfil','sum']))
#total_cobranca = total_cobranca.rename(columns={'sum':'Total'})
total_cobranca = total_cobranca.merge(sap,how='inner',left_on=['CNPJ Contraparte_x'],right_on=['CNPJ'])
total_cobranca=total_cobranca.drop_duplicates().copy().reset_index(drop = True)
total_cobranca.to_csv('total.csv',index = False)
total_cobranca = total_cobranca.drop(columns=['CNPJ Contraparte_x'])
#total_cobranca.head()

#total = pd.DataFrame(bf.groupby(['Parte - Perfil','Parte - Código do Perfil','Contraparte - Perfil','ContraParte - Código do Perfil'])["COBRAR\nR$"].agg(['sum']))
#print(total)
pre_total = pd.DataFrame(total_cobranca.groupby(["CNPJ"])["Total"].agg(['sum']))
pre_total.head()
print(pre_total)

                         sum
CNPJ                        
07.685.694/0001-97    521.07
08.573.833/0001-53    202.66
10.671.322/0001-16     43.16
13.459.301/0001-20   3288.44
19.125.927/0001-86  17642.92
26.940.979/0001-71    395.77
28.758.086/0001-35   1254.10
30.840.548/0001-00   1973.07
32.023.463/0001-65   2221.29
34.801.306/0001-69     18.49
34.805.807/0001-13     18.49
34.818.458/0001-74     18.49
34.818.597/0001-06     18.49
34.841.996/0001-80     18.49
37.543.498/0001-49    927.28


In [23]:
merged2 = merged2.rename(columns={'sum':'Total'})
merged2['Total'] = merged2['Total'].round(2)
merged2 = merged2.rename(columns={'COBRAR\nR$':'Cobrar'})
merged2.to_csv('help.csv',index = False)

In [24]:
bf3 = pd.DataFrame(merged2.groupby(['Parte - Perfil','Parte - Código do Perfil','Contraparte - Perfil','ContraParte - Código do Perfil','Mês','Total'])["Cobrar"].agg(['sum']))
merged3 = pd.merge(df,bf3,how='inner',on=['Parte - Perfil','Parte - Código do Perfil','Contraparte - Perfil','ContraParte - Código do Perfil','Mês'])

merged3['sum'] = merged3['sum'].round(2)
print(merged3.head())
merged3.to_csv('help2.csv',index = False)

           Parte - Perfil  Parte - Código do Perfil      Contraparte - Perfil  \
0  CASA DOS VENTOS COM I5                     27977  DEAL COMERCIALIZADORA I5   
1  CASA DOS VENTOS COM I5                     27977            SOLENERGIAS I5   
2  CASA DOS VENTOS COM I5                     27977              STATKRAFT I5   
3  CASA DOS VENTOS COM I5                     27977             ESFERA COM I5   
4  CASA DOS VENTOS COM I5                     27977                 NEWCOM I5   

   ContraParte - Código do Perfil      Mês          CNPJ Parte  \
0                            3726  mai.-24  33.933.760/0001-00   
1                            4571  mai.-24  33.933.760/0001-00   
2                           11894  mai.-24  33.933.760/0001-00   
3                           70304  mai.-24  33.933.760/0001-00   
4                           78922  mai.-24  33.933.760/0001-00   

     CNPJ Contraparte      sum  
0  10.671.322/0001-16    43.16  
1  13.459.301/0001-20  3288.44  
2  08.573.833/000

In [25]:
bf4 = merged3

#bf4 = bf4.drop(columns=['Parte - Código do Perfil','ContraParte - Código do Perfil'])

bf4.head()
bf4.to_csv('help3.csv',index = False)


In [14]:
bf5=pre_total

In [19]:
base_final = bf5.merge(sap,how='inner',left_on=['CNPJ'],right_on=['CNPJ'])

base_final.head()

base_final.drop(columns='CNPJ')

base_final = base_final[['Nome','CNPJ','Endereço','Bairro','Cidade','UF','CEP','Telefone','sum']]

base_final['vencimento'] = (dt.datetime.today()+dt.timedelta(days=30)).strftime("%d/%m/%Y")

base_final['base'] = 'Fortaleza - CE, ' + dt.datetime.today().strftime("%d/%m/%Y")

#base_final['file_name'] = base_final['Mês'] + base_final['Nome'].str.slice(0,4) 

base_final.to_csv('bf.csv',index = False)

base_final.head()

,Nome,CNPJ,Endereço,Bairro,Cidade,UF,CEP,Telefone,sum,vencimento,base
0,ENERGISA COMERCIALIZADORA LTDA,07.685.694/0001-97,"Pr Botafogo 228, Sl 1301",Botafogo,RIO DE JANEIRO,RJ,22250-906,21212269,521.07,06/09/2024,"Fortaleza - CE, 07/08/2024"
1,STATKRAFT ENERGIA DO BRAS,08.573.833/0001-53,"Rod Jose Carlos Daux, 5500, Sl 326",Saco Grande,FLORIANOPOLIS,SC,88032-005,48387771,202.66,06/09/2024,"Fortaleza - CE, 07/08/2024"
2,DEAL COMERCIALIZADORA DE,10.671.322/0001-16,"Al Campinas 802, And 15",Jardim Paulista,SAO PAULO,SP,01404-200,11359408,43.16,06/09/2024,"Fortaleza - CE, 07/08/2024"
3,EQUATORIAL RENOVAVEIS S.A,13.459.301/0001-20,"Av das Nacoes Unidas 14171, And 15",Vila Gertrudes,SAO PAULO,SP,04794-000,11493540,3288.44,06/09/2024,"Fortaleza - CE, 07/08/2024"
4,COPEL COMERCIALIZACAO S.A,19.125.927/0001-86,R Coronel Dulcidio 800,Batel,CURITIBA,PR,80420-170,41333141,17642.92,06/09/2024,"Fortaleza - CE, 07/08/2024"


In [23]:


    
print(total_cobranca.iloc[34]["ContraParte - Código do Perfil"])
    


25387


In [26]:
list_meses = []
base_mes = bf4[["Mês","CNPJ Contraparte"]].sort_values(by='CNPJ Contraparte')
print(base_mes)
for i in range(len(base_final)):
    meses = ""
    meses_total = base_mes[["Mês","CNPJ Contraparte"]][base_mes["CNPJ Contraparte"]==base_final.iloc[i]["CNPJ"]]
    print(meses_total)
    for j in range(len(meses_total)):
        meses = meses + str(meses_total.iloc[j]["Mês"]) + " | "
       
    list_meses.append(meses)   
    print(meses)   
        
        
        
print(list_meses)      

        Mês    CNPJ Contraparte
14  mai.-24  07.685.694/0001-97
2   mai.-24  08.573.833/0001-53
0   mai.-24  10.671.322/0001-16
1   mai.-24  13.459.301/0001-20
13  mai.-24  19.125.927/0001-86
3   mai.-24  26.940.979/0001-71
4   mai.-24  28.758.086/0001-35
5   mai.-24  30.840.548/0001-00
6   mai.-24  32.023.463/0001-65
12  mai.-24  34.801.306/0001-69
11  mai.-24  34.805.807/0001-13
9   mai.-24  34.818.458/0001-74
10  mai.-24  34.818.597/0001-06
8   mai.-24  34.841.996/0001-80
7   mai.-24  37.543.498/0001-49
        Mês    CNPJ Contraparte
14  mai.-24  07.685.694/0001-97
mai.-24 | 
       Mês    CNPJ Contraparte
2  mai.-24  08.573.833/0001-53
mai.-24 | 
       Mês    CNPJ Contraparte
0  mai.-24  10.671.322/0001-16
mai.-24 | 
       Mês    CNPJ Contraparte
1  mai.-24  13.459.301/0001-20
mai.-24 | 
        Mês    CNPJ Contraparte
13  mai.-24  19.125.927/0001-86
mai.-24 | 
       Mês    CNPJ Contraparte
3  mai.-24  26.940.979/0001-71
mai.-24 | 
       Mês    CNPJ Contraparte
4  mai.-24  28.

In [29]:
bf_total = base_final

bf_total['Meses'] = list_meses

bf_total['vencimento'] = (dt.datetime.today()+dt.timedelta(days=30)).strftime("%d/%m/%Y")

bf_total['base'] = 'Fortaleza - CE, ' + dt.datetime.today().strftime("%d/%m/%Y")

bf_total['file_name'] = bf_total['Nome'].str.slice(0,4) 

bf_total = bf_total.rename(columns={'sum':'Total'})

bf_total.head()

,Nome,CNPJ,Endereço,Bairro,Cidade,UF,CEP,Telefone,Total,vencimento,base,Meses,file_name
0,ENERGISA COMERCIALIZADORA LTDA,07.685.694/0001-97,"Pr Botafogo 228, Sl 1301",Botafogo,RIO DE JANEIRO,RJ,22250-906,21212269,521.07,06/09/2024,"Fortaleza - CE, 07/08/2024",mai.-24 |,ENER
1,STATKRAFT ENERGIA DO BRAS,08.573.833/0001-53,"Rod Jose Carlos Daux, 5500, Sl 326",Saco Grande,FLORIANOPOLIS,SC,88032-005,48387771,202.66,06/09/2024,"Fortaleza - CE, 07/08/2024",mai.-24 |,STAT
2,DEAL COMERCIALIZADORA DE,10.671.322/0001-16,"Al Campinas 802, And 15",Jardim Paulista,SAO PAULO,SP,01404-200,11359408,43.16,06/09/2024,"Fortaleza - CE, 07/08/2024",mai.-24 |,DEAL
3,EQUATORIAL RENOVAVEIS S.A,13.459.301/0001-20,"Av das Nacoes Unidas 14171, And 15",Vila Gertrudes,SAO PAULO,SP,04794-000,11493540,3288.44,06/09/2024,"Fortaleza - CE, 07/08/2024",mai.-24 |,EQUA
4,COPEL COMERCIALIZACAO S.A,19.125.927/0001-86,R Coronel Dulcidio 800,Batel,CURITIBA,PR,80420-170,41333141,17642.92,06/09/2024,"Fortaleza - CE, 07/08/2024",mai.-24 |,COPE


In [76]:
get_and_modify_xl(base_final)

0

In [30]:
def formatar_coluna(input_float):
        pre_string = f"{input_float:,.2f}"
        out_string = pre_string.replace(",",";").replace(".",",").replace(";",".")

        return out_string
    

bf_total['Total']=bf_total.apply(lambda x: formatar_coluna(float(x["Total"])),axis=1) 
print(bf_total['Total'])

0        521,07
1        202,66
2         43,16
3      3.288,44
4     17.642,92
5        395,77
6      1.254,10
7      1.973,07
8      2.221,29
9         18,49
10        18,49
11        18,49
12        18,49
13        18,49
14       927,28
Name: Total, dtype: object


In [31]:
from datetime import date, datetime

date_today = date.today()
year_date = date_today.strftime( "%Y")
day_date = date_today.strftime("%d")
mounth_date = int(date_today.strftime("%m"))
mounth = ['Janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']
print(mounth_date)

8


In [32]:
for i in range(len(bf_total)):
    
        from datetime import date, datetime
        from reportlab.pdfgen import canvas
        from reportlab.lib.pagesizes import A4
        from reportlab.lib.styles import getSampleStyleSheet
        from reportlab.lib import colors
        from reportlab.platypus import Table
        from reportlab.lib.units import inch
        from reportlab.pdfbase import pdfmetrics
        from reportlab.pdfbase.ttfonts import TTFont

        pdfmetrics.registerFont(TTFont('Calibri', 'static/fonts/calibri.ttf'))
        pdfmetrics.registerFont(TTFont('CalibriBold', 'static/fonts/calibrib.ttf'))
        pdfmetrics.registerFont(TTFont('CalibriIt', 'static/fonts/calibrii.ttf'))
        pdfmetrics.registerFont(TTFont('Calibriz', 'static/fonts/calibriz.ttf'))

                
        pdf = canvas.Canvas( str(i) + "_ND" + "-" + str(bf_total['file_name'].iloc[i]) + ".pdf", pagesize=(600,900))

        def init_page(pdf):
                    
                    pdf.setFillColorRGB(0/256,0/256,0/256)
                    pdf.setStrokeColorRGB(0/256,0/256,0/256)
                    pdf.setFont("CalibriBold", 12) #choose your font type and font size
                    pdf.drawString(230,850,"Casa dos Ventos Comercializadora de Energia S.A.")
                    pdf.drawImage("static/Logos/Simbolo_BRANCO.jpg",20,835,height=45,width=160)
                    pdf.setFillColorRGB(191/256,191/256,191/256)
                    pdf.rect(21, 800, 559 , 20, fill=1,stroke=0)
                    pdf.rect(21, 490, 559 , 20, fill=1,stroke=0)
                    pdf.rect(21,305,559,20,fill=1,stroke=0)
                    pdf.setFillColorRGB(0/256,0/256,0/256)
                    pdf.drawString(240,805,"Nota de Débito - 001")
                    pdf.rect(20, 685, 560 , 135)
                    pdf.rect(20, 530, 560 , 135)
                    pdf.rect(20, 350, 560 , 160)
                    pdf.drawString(250,495,"Dados da ND")
                    pdf.drawString(245,310,"Dados Bancários")
                    pdf.line(190,489,190,350)
                    pdf.line(20,489,580,489)
                    pdf.line(20,410,580,410)
                    pdf.line(20,390,580,390)
                    pdf.line(20,370,580,370)
                    pdf.rect(20,225,560,100)
                    pdf.line(190,304,190,225)
                    pdf.line(20,304,580,304)
                    pdf.line(20,285,580,285)
                    pdf.line(20,265,580,265)
                    pdf.line(20,245,580,245)
                    pdf.setFont("Calibri", 12)
                    pdf.setFillColorRGB(0/256,0/256,0/256)
                    pdf.setStrokeColorRGB(0/256,0/256,0/256)
                    pdf.rect(1,205,598,694)
                    pdf.drawString(200,180,"Fortaleza - CE, " + day_date + " de " + mounth[mounth_date-1] + " de " + year_date)
                    pdf.line(20,160,560,160)
                    pdf.drawString(180,140,"Casa dos Ventos Comercializadora de Energia S.A.")
                    
                    return pdf   
                
        def Q1(pdf):
            pdf.setFont("Calibri", 12)
            pdf.setFillColorRGB(0/256,0/256,0/256)
            pdf.setStrokeColorRGB(0/256,0/256,0/256)
            pdf.drawString(21,787," CNPJ")
            pdf.drawString(190,787,"33.933.760/0001-00")
            pdf.drawString(21,772," Endereço")
            pdf.drawString(190,772,"AVENIDA DESEMBARGADOR MOREIRA, 1300 - SALA 902 T-NORTE")
            pdf.drawString(21,756," Bairro")
            pdf.drawString(190,756,"ALDEOTA")
            pdf.drawString(21,740," Cidade")
            pdf.drawString(190,740,"FORTALEZA")
            pdf.drawString(21,724," UF")
            pdf.drawString(190,724,"CEARÁ")
            pdf.drawString(21,708," CEP")
            pdf.drawString(190,708,"60.170-002")
            pdf.drawString(21,692," Telefone")
            pdf.drawString(190,692,"(85) 3034.9774")
                    
            
            return pdf

        def Q2(pdf,base_final, i):
            pdf.setFont("Calibri", 12)
            pdf.setFillColorRGB(0/256,0/256,0/256)
            pdf.setStrokeColorRGB(0/256,0/256,0/256)
            pdf.drawString(21,653," Razão social")
            pdf.setFont("CalibriBold", 12)
            pdf.drawString(190,653,str(bf_total.iloc[i]['Nome']))
            pdf.setFont("Calibri", 12)
            pdf.drawString(21,638," CNPJ")
            pdf.drawString(190,638, str(bf_total.iloc[i]['CNPJ']))
            pdf.drawString(21,622," Endereço")
            pdf.drawString(190,622,str(bf_total.iloc[i]['Endereço']))
            pdf.drawString(21,606," Bairro")
            pdf.drawString(190,606,str(bf_total.iloc[i]['Bairro']))
            pdf.drawString(21,590," Cidade")
            pdf.drawString(190,590,str(bf_total.iloc[i]['Cidade']))
            pdf.drawString(21,574," UF")
            pdf.drawString(190,574,str(bf_total.iloc[i]['UF']))
            pdf.drawString(21,558," CEP")
            pdf.drawString(190,558,str(bf_total.iloc[i]['CEP']))
            pdf.drawString(21,542," Telefone")
            pdf.drawString(190,542,str(bf_total.iloc[i]['Telefone']))
                    
            
            return pdf

        def Q3(pdf,bf_total, i):
            
            pdf.drawString(21,475," Descrição do Objeto")
            pdf.setFont("Calibri", 11)
            pdf.drawString(190,475," Nota de Débito correspondente ao valor referente à ressarcimento a ser")
            pdf.drawString(190,460," efetuado a Casa dos Ventos Comercializadora de Energia S.A. referente à entrega ")
            pdf.drawString(190,445," inferior do desconto contratado, conforme demonstrado no cálculo em anexo.  ")
            pdf.drawString(190,430," CNPJ para pagamento 33.933.760/0001-00")
            pdf.setFont("Calibri", 12)
            pdf.drawString(21,395," Competência")
            pdf.setFont("Calibri", 10)
            pdf.drawString(190,395," " +str(bf_total.iloc[i]['Meses']))
            pdf.setFont("Calibri", 12)
            pdf.drawString(21,375," Valor")
            width = pdf._pagesize[0]
            padding = 25
            pdf.drawRightString(width - padding, 375,"R$ " + str(bf_total.iloc[i]['Total']))
            pdf.drawString(21,355," Data de Vencimento")
            pdf.drawString(190,355," " +str(bf_total.iloc[i]['vencimento']))
            
            return pdf

        def Q4(pdf):
            pdf.drawString(21,290," Cnpj")
            pdf.drawString(190,290," 33.933.760/0001-00 - Matriz")
            pdf.drawString(21,270," Banco")
            pdf.drawString(190,270," SANTANDER - 033")
            pdf.drawString(21,250," Agência")
            pdf.drawString(190,250," 2271")
            pdf.drawString(21,230," Conta")
            pdf.drawString(190,230," 13088857-6")
            return pdf


        pdf = init_page(pdf)    
        pdf = Q1(pdf)  
        pdf = Q2(pdf,bf_total,i)  
        pdf = Q3(pdf,bf_total,i)  
        pdf = Q4(pdf) 
        pdf.save()
    